# 🎬 EXO_03_CONTROL — Debug Notebook

```
╔══════════════════════════════════════════════════════════════════════════════╗
║                 SCENOGRAPHY DOCK — CONTROL PANEL                             ║
║              Test et Debug Construction Environnements                       ║
╚══════════════════════════════════════════════════════════════════════════════╝
```

**Mission:** Tester unitairement les modules de construction d'environnements 3D

## 1. Configuration

In [ ]:
import os
import sys
import json
from pathlib import Path

# Configuration paths - MODIFIER SELON VOTRE ENVIRONNEMENT
DRIVE_ROOT = Path("/content/drive/MyDrive/DRIVE_EXODUS_V2")
# DRIVE_ROOT = Path("/home/EXODUS-V2")  # Local dev

UNIT_ROOT = DRIVE_ROOT / "03_SCENOGRAPHY_DOCK"
CODEBASE = UNIT_ROOT / "CODEBASE"
IN_MAPS = UNIT_ROOT / "IN_MAPS"
OUT_ENVIRONMENTS = UNIT_ROOT / "OUT_ENVIRONMENTS"

# Ajouter CODEBASE au path
sys.path.insert(0, str(CODEBASE))

print(f"Drive Root: {DRIVE_ROOT}")
print(f"Unit Root: {UNIT_ROOT}")
print(f"Exists: {UNIT_ROOT.exists()}")

## 2. Vérifier Structure

In [ ]:
# Vérifier les dossiers
folders = [
    CODEBASE,
    IN_MAPS,
    IN_MAPS / "hdri_library",
    IN_MAPS / "environment_assets",
    OUT_ENVIRONMENTS
]

for folder in folders:
    status = "✓" if folder.exists() else "✗"
    print(f"{status} {folder.relative_to(DRIVE_ROOT)}")
    if not folder.exists():
        folder.mkdir(parents=True, exist_ok=True)
        print(f"  → Créé")

## 3. Créer PRODUCTION_PLAN Test

In [ ]:
# Plan de test avec environnements variés
test_plan = {
    "metadata": {
        "source_video": "test_video.mp4",
        "cortex_version": "2.0",
        "analysis_date": "2026-02-03"
    },
    "scenes": [
        {
            "scene_id": 1,
            "description": "Scène de rue urbaine nocturne",
            "environment": {
                "type": "urban_street",
                "description": "Rue de ville avec néons",
                "lighting_mood": "neon",
                "props": ["streetlight", "bench", "trash_can"]
            }
        },
        {
            "scene_id": 2,
            "description": "Intérieur bureau",
            "environment": {
                "type": "indoor",
                "description": "Bureau moderne",
                "lighting_mood": "natural",
                "props": ["table", "chair"]
            }
        },
        {
            "scene_id": 3,
            "description": "Studio photo",
            "environment": {
                "type": "studio",
                "description": "Studio avec cyclorama blanc",
                "lighting_mood": "studio",
                "props": []
            }
        }
    ]
}

# Sauvegarder
plan_path = IN_MAPS / "PRODUCTION_PLAN_TEST.JSON"
with open(plan_path, 'w', encoding='utf-8') as f:
    json.dump(test_plan, f, indent=2, ensure_ascii=False)

print(f"Plan de test créé: {plan_path}")
print(f"Scènes: {len(test_plan['scenes'])}")

## 4. Test Dry-Run

In [ ]:
# Test dry-run du wrapper principal
!python {CODEBASE}/EXO_03_SCENOGRAPHY.py \
    --drive-root {DRIVE_ROOT} \
    --production-plan {plan_path} \
    --dry-run \
    -v

## 5. Vérifier Blender

In [ ]:
# Vérifier Blender installation
BLENDER_PATH = DRIVE_ROOT / "EXODUS_AI_MODELS" / "blender-4.0.0-linux-x64" / "blender"

if BLENDER_PATH.exists():
    print(f"✓ Blender trouvé: {BLENDER_PATH}")
    !{BLENDER_PATH} --version
else:
    print(f"✗ Blender NON TROUVÉ: {BLENDER_PATH}")
    print("\nPour installer Blender:")
    print("1. Téléchargez Blender 4.0 Linux x64 portable")
    print(f"2. Extrayez dans: {BLENDER_PATH.parent}")

## 6. Lister HDRi Disponibles

In [ ]:
hdri_path = IN_MAPS / "hdri_library"

if hdri_path.exists():
    hdri_files = list(hdri_path.glob("*.hdr")) + list(hdri_path.glob("*.exr"))
    print(f"HDRi trouvés: {len(hdri_files)}")
    for f in hdri_files:
        size_mb = f.stat().st_size / (1024*1024)
        print(f"  • {f.name} ({size_mb:.1f} MB)")
else:
    print(f"Dossier HDRi non trouvé: {hdri_path}")
    print("\nPour ajouter des HDRi:")
    print("1. Téléchargez des HDRi depuis Poly Haven, HDRI Haven, etc.")
    print(f"2. Placez les fichiers .hdr ou .exr dans: {hdri_path}")
    print("3. Nommez-les selon le mood: neon.hdr, dramatic.hdr, natural.hdr, studio.hdr")

## 7. Test Construction Simple (si Blender disponible)

In [ ]:
# Exécution réelle (requiert Blender)
if BLENDER_PATH.exists():
    print("Lancement construction scène de test...")
    !python {CODEBASE}/EXO_03_SCENOGRAPHY.py \
        --drive-root {DRIVE_ROOT} \
        --production-plan {plan_path} \
        --blender-path {BLENDER_PATH} \
        --scene-ids 3 \
        -v
else:
    print("⚠ Blender non disponible, skip exécution réelle")

## 8. Vérifier Outputs

In [ ]:
# Vérifier les fichiers générés
if OUT_ENVIRONMENTS.exists():
    files = list(OUT_ENVIRONMENTS.glob("*"))
    print(f"Fichiers dans OUT_ENVIRONMENTS: {len(files)}")
    for f in files:
        size_mb = f.stat().st_size / (1024*1024) if f.is_file() else 0
        print(f"  • {f.name} ({size_mb:.2f} MB)")
        
    # Afficher rapport si existe
    report_path = OUT_ENVIRONMENTS / "scenography_report.json"
    if report_path.exists():
        print("\n=== RAPPORT ===")
        with open(report_path, 'r') as f:
            report = json.load(f)
        print(f"Status: {report.get('status')}")
        print(f"Scènes: {report.get('summary', {}).get('scenes_built', 0)}")
else:
    print("Dossier OUT_ENVIRONMENTS vide ou inexistant")

---
*EXODUS SYSTEM — Frégate 03_SCENOGRAPHY v1.0.0*